### David Nahm Moloco 

In [3]:
import sqlite3
import pandas as pd

In [6]:
# Read in data as Pandas dataframe and connect to sqlite database to conduct SQL queries
df = pd.read_csv('Adops & Data Scientist Sample Data - Q1 Analytics.csv')
conn = sqlite3.connect('Moloco.db')
df.to_sql('moloco', conn)

In [7]:
# Check to make sure database set up correctly
sql_string = 'SELECT * FROM moloco LIMIT 1'
out = pd.read_sql(sql_string, conn)
out

,index,ts,user_id,country_id,site_id
0,0,2019-02-01 00:01:24,LC36FC,TL6,N0OTG


#### 1) Consider only the rows with country_id = "BDV" (there are 844 such rows). For each site_id, we can compute the number of unique user_id's found in these 844 rows. Which site_id has the largest number of unique users? And what's the number?

I first selected only the "BDV" country_id, then grouped by the different site_ids. I then counted the number of distinct users for each site_id. 

In [12]:
sql_string = 'SELECT site_id, COUNT(DISTINCT user_id) AS num_users \
                FROM moloco \
                WHERE country_id \
                LIKE "BDV" \
                GROUP BY site_id;'
out = pd.read_sql(sql_string, conn)
out

,site_id,num_users
0,3POLC,2
1,5NPAU,544
2,N0OTG,90


#### 2) Between 2019-02-03 00:00:00 and 2019-02-04 23:59:59, there are four users who visited a certain site more than 10 times. Find these four users & which sites they (each) visited more than 10 times. (Simply provides four triples in the form (user_id, site_id, number of visits) in the box below.)

I first filtered out data from outside of the desired time range. Then, I grouped by user_id and site_id pairings and counted which of these pairings occurred more than 10 times. 

In [11]:
sql_string = 'SELECT user_id, site_id, COUNT(*) AS num_visits \
                FROM moloco \
                WHERE ts > "2019-02-03 00:00:00" AND ts < "2019-02-04 23:59:59" \
                GROUP BY user_id, site_id \
                HAVING COUNT(*) > 10;'
out = pd.read_sql(sql_string, conn)
out

,user_id,site_id,num_visits
0,LC06C3,N0OTG,25
1,LC3A59,N0OTG,26
2,LC3C7E,3POLC,15
3,LC3C9D,N0OTG,17


#### 3) For each site, compute the unique number of users whose last visit (found in the original data set) was to that site. For instance, user "LC3561"'s last visit is to "N0OTG" based on timestamp data. Based on this measure, what are top three sites? (hint: site "3POLC" is ranked at 5th with 28 users whose last visit in the data set was to 3POLC; simply provide three pairs in the form (site_id, number of users).)

I first find the record containing the last visit for each user. Then, I group by site_id and count the number of visits per site and order in descending order. The answer to the question is the first 3 records in the results, but I left the other results as a sanity check. 

In [13]:
sql_string = 'SELECT site_id, COUNT(*) as num_users \
                FROM (SELECT m1.* \
                        FROM moloco AS m1 \
                        LEFT OUTER JOIN moloco AS m2 ON m1.user_id = m2.user_id AND m1.ts < m2.ts \
                        WHERE m2.user_id IS NULL) \
                AS m \
                GROUP BY site_id \
                ORDER BY num_users DESC;'
out = pd.read_sql(sql_string, conn)
out

,site_id,num_users
0,5NPAU,992
1,N0OTG,561
2,QGO3G,289
3,GVOFK,42
4,3POLC,28
5,RT9Z6,2
6,EUZ/Q,1
7,JSUUP,1


#### 4) For each user, determine the first site he/she visited and the last site he/she visited based on the timestamp data. Compute the number of users whose first/last visits are to the same website. What is the number?

I first find the corresponding record for each user's first and last site visited. Then, I join these tables on user_id and site_id to get the users who visited the same site on the first and last visit. Finally, I count the number of users. 

In [10]:
sql_string = 'SELECT COUNT(*) AS num_users \
                FROM (SELECT m1.* \
                        FROM moloco as m1 \
                        LEFT OUTER JOIN moloco as m2 \
                        ON (m1.user_id = m2.user_id AND m1.ts < m2.ts) \
                        WHERE m2.user_id IS NULL) \
                AS m1 \
                INNER JOIN (SELECT m1.* \
                                FROM moloco as m1 \
                                LEFT OUTER JOIN moloco as m2 \
                                ON (m1.user_id = m2.user_id AND m1.ts > m2.ts) \
                                WHERE m2.user_id IS NULL) \
                AS m2 \
                ON m1.user_id = m2.user_id AND m1.site_id = m2.site_id;'
out = pd.read_sql(sql_string, conn)
out

,num_users
0,1670
